In [15]:
import sys
import os

# 🔹 상위 폴더(`preprocess/`)를 sys.path에 추가
sys.path.append(os.path.abspath(".."))

# 🔹 utils 모듈 import
from utils import output_as_dataset, add_metadata, extract_original_info, renew_episode_info, add_stage_2

In [16]:
import json

with open("/HabitatLLM/preprocess/sample/object_semantics.json", "r") as f:
    tuned_episodes = json.load(f)['episodes']

In [17]:
data_path = "/HabitatLLM/data/datasets/partnr_episodes/v0_0/val.json"

used_episodes = []
for d in tuned_episodes:
    used_episodes.append(d['episode_id'])

In [18]:
list(set(used_episodes))

['577', '398', '576', '399', '401', '575', '574', '23', '393', '397', '578']

In [19]:
with open(data_path, "r") as f:
    original_data = json.load(f)['episodes']

In [20]:
original_informations = []
for ep_id in used_episodes:
    for od in original_data:
        if od['episode_id'] == ep_id:
            original_info = {
                "episode_id": ep_id,
                "instruction": od['instruction'],
                "data_path": data_path
            }
            original_informations.append(original_info)
        
        

In [21]:
tuned_episodes[0]['instruction']

'Help me move some things around. Put the sponge on the bedside table. Move the action figure to the stand in the living room. Put the plant saucer on the kitchen counter. The sponge is green, the figure is a Mario action figure, and the plant saucer is green and holds potted plants. Those items belong to me.'

In [22]:
original_informations

[{'episode_id': '23',
  'instruction': 'Help me move some things around. Put the sponge on the bedside table. Move the action figures to the stand in the living room. Put the plant saucer on the kitchen counter.',
  'data_path': '/HabitatLLM/data/datasets/partnr_episodes/v0_0/val.json'},
 {'episode_id': '393',
  'instruction': 'Move the lamp to the bedroom table. Move the statue to the living room shelves.',
  'data_path': '/HabitatLLM/data/datasets/partnr_episodes/v0_0/val.json'},
 {'episode_id': '397',
  'instruction': 'Help me set up some visuals on the shelves. Move the picture frame and lamp there. The book can stay where it is.',
  'data_path': '/HabitatLLM/data/datasets/partnr_episodes/v0_0/val.json'},
 {'episode_id': '398',
  'instruction': 'Can we move the candle and vase from the living room table to the bedroom chest of drawers?',
  'data_path': '/HabitatLLM/data/datasets/partnr_episodes/v0_0/val.json'},
 {'episode_id': '399',
  'instruction': 'Move the jug and cup from the 

In [23]:
for i, ep in enumerate(tuned_episodes):
    for info in original_informations:
        if ep['episode_id'] == info['episode_id']:
            tuned_episodes[i]['original_data_info'] = info

In [24]:
tuned_episodes[0]['original_data_info']

{'episode_id': '23',
 'instruction': 'Help me move some things around. Put the sponge on the bedside table. Move the action figures to the stand in the living room. Put the plant saucer on the kitchen counter.',
 'data_path': '/HabitatLLM/data/datasets/partnr_episodes/v0_0/val.json'}

In [25]:
from copy import deepcopy

episode_copy = deepcopy(tuned_episodes)
stage_1_episodes = tuned_episodes[:11]
stage_2_episodes = tuned_episodes[11:]
stage_1_episodes = renew_episode_info(stage_1_episodes, 0)

In [26]:
stage_1_episodes = add_metadata(stage_1_episodes)
final_episodes = add_stage_2(stage_1_episodes)

In [27]:
from copy import deepcopy

# ✅ final_episodes를 깊은 복사하여 변경할 리스트 생성 (원본 유지)
updated_final_episodes = deepcopy(final_episodes)

# ✅ episode_copy를 두 개로 분리하여 사용
episode_copy_1 = episode_copy[:11]  # 원본의 앞부분 (0~10)
print(episode_copy_1[0]['instruction'])
episode_copy_2 = episode_copy[11:22]  # 원본의 뒷부분 (11~21)
print(episode_copy_2[0]['instruction'])

print("==========================================")
# ✅ stage_1 (0~10)에 대한 instruction 반영
for i in range(11):  # 0~10
    print(episode_copy_1[i]['instruction'])
    updated_final_episodes[i]['instruction'] = episode_copy_1[i]['instruction']

print("==========================================")
# ✅ stage_2 (11~21)에 대한 instruction 반
for i in range(11, 22):  # 11~21
    print(episode_copy_2[i-11]['instruction'])
    updated_final_episodes[i]['instruction'] = episode_copy_2[i - 11]['instruction']  # 11부터 맞추기 위해 조정

print("==========================================")
# ✅ 변경 사항 확인
print(updated_final_episodes[1]['instruction'])  # 변경된 값 확인
print(updated_final_episodes[12]['instruction'])  # 원본 유지 확인

Help me move some things around. Put the sponge on the bedside table. Move the action figure to the stand in the living room. Put the plant saucer on the kitchen counter. The sponge is green, the figure is a Mario action figure, and the plant saucer is green and holds potted plants. Those items belong to me.
Help me move some things around. Put my sponge on the bedside table. Move my action figure to the stand in the living room. Put my plant saucer on the kitchen counter.
Help me move some things around. Put the sponge on the bedside table. Move the action figure to the stand in the living room. Put the plant saucer on the kitchen counter. The sponge is green, the figure is a Mario action figure, and the plant saucer is green and holds potted plants. Those items belong to me.
Move the lamp to the bedroom table. Move the statue to the living room shelves. The statue is holding a towel. The statue is mine.
Help me set up some visuals on the shelves. Move the picture frame and lamp there

In [28]:
from copy import deepcopy

# ✅ final_episodes를 깊은 복사하여 변경할 리스트 생성 (원본 유지)
updated_final_episodes = deepcopy(final_episodes)

# ✅ episode_copy를 두 개로 분리하여 사용 (정확한 분할 확인)
episode_copy_1 = deepcopy(episode_copy[:11])  # 원본의 앞부분 (0~10)
episode_copy_2 = deepcopy(episode_copy[11:22])  # 원본의 뒷부분 (11~21)

# ✅ 분리된 리스트 확인
print("=== episode_copy_1 ===")
for i, ep in enumerate(episode_copy_1):
    print(f"{i}: {ep['episode_id']} -> {ep['instruction']}")

print("\n=== episode_copy_2 ===")
for i, ep in enumerate(episode_copy_2):
    print(f"{i+11}: {ep['episode_id']} -> {ep['instruction']}")

# ✅ stage_1 (0~10)에 대한 instruction 반영
for i in range(11):  # 0~10
    updated_final_episodes[i]['instruction'] = episode_copy_1[i]['instruction']

# ✅ stage_2 (11~21)에 대한 instruction 반영
for i in range(11, 22):  # 11~21
    updated_final_episodes[i]['instruction'] = episode_copy_2[i - 11]['instruction']  # 11부터 맞추기 위해 조정

# ✅ 변경 사항 확인
print("\n=== Updated final_episodes ===")
print(updated_final_episodes[0]['instruction'])  # 변경된 값 확인
print(updated_final_episodes[11]['instruction'])  # 원본 유지 확인

=== episode_copy_1 ===
0: 23 -> Help me move some things around. Put the sponge on the bedside table. Move the action figure to the stand in the living room. Put the plant saucer on the kitchen counter. The sponge is green, the figure is a Mario action figure, and the plant saucer is green and holds potted plants. Those items belong to me.
1: 393 -> Move the lamp to the bedroom table. Move the statue to the living room shelves. The statue is holding a towel. The statue is mine.
2: 397 -> Help me set up some visuals on the shelves. Move the picture frame and lamp there. The book can stay where it is. The lamp is pineapple-shaped and has a black base. The lamp is mine.
3: 398 -> Can we move the candle and vase from the living room table to the bedroom chest of drawers? The vase is brown, has a tall, narrow shape. The vase is mine.
4: 399 -> Move the jug and cup from the kitchen counter to the living room table. Place them next to each other. The cup is white, has a purple flower design. 

In [29]:
print("=== episode_copy_1 ===")
for i, ep in enumerate(episode_copy_1):
    print(f"{i}: {ep['episode_id']} -> {ep['instruction']}")

print("\n=== episode_copy_2 ===")
for i, ep in enumerate(episode_copy_2):
    print(f"{i+11}: {ep['episode_id']} -> {ep['instruction']}")

=== episode_copy_1 ===
0: 23 -> Help me move some things around. Put the sponge on the bedside table. Move the action figure to the stand in the living room. Put the plant saucer on the kitchen counter. The sponge is green, the figure is a Mario action figure, and the plant saucer is green and holds potted plants. Those items belong to me.
1: 393 -> Move the lamp to the bedroom table. Move the statue to the living room shelves. The statue is holding a towel. The statue is mine.
2: 397 -> Help me set up some visuals on the shelves. Move the picture frame and lamp there. The book can stay where it is. The lamp is pineapple-shaped and has a black base. The lamp is mine.
3: 398 -> Can we move the candle and vase from the living room table to the bedroom chest of drawers? The vase is brown, has a tall, narrow shape. The vase is mine.
4: 399 -> Move the jug and cup from the kitchen counter to the living room table. Place them next to each other. The cup is white, has a purple flower design. 

In [30]:
# for i, ep in enumerate(final_episodes[:11]):
#     for ep_o in stage_1_episodes_copy:
#         if ep['original_data_info']['episode_id'] == ep_o['episode_id']:
#             final_episodes[i]['instruction'] = ep_o['instruction']

# for i, ep in enumerate(updated_final_episodes[11:]):
#     for ep_o in stage_2_episodes:
#         if ep['original_data_info']['episode_id'] == ep_o['episode_id']:
#             final_episodes[11+i]['instruction'] = ep_o['instruction']

In [31]:
updated_final_episodes[0]['instruction']

'Help me move some things around. Put the sponge on the bedside table. Move the action figure to the stand in the living room. Put the plant saucer on the kitchen counter. The sponge is green, the figure is a Mario action figure, and the plant saucer is green and holds potted plants. Those items belong to me.'

In [32]:
output_as_dataset(updated_final_episodes, "object_semantics_final.json")

Dataset saved to object_semantics_final.json
